# Init data


Lancer la commande `docker cp allData.zip jupyter:/home/jovyan/` en premier lieu.

In [ ]:
# Commandes à lancer dans le notebook (WSL & Mac) 
!ls
!unzip -o allData.zip 
!ls

In [ ]:
minio_ip_address = "minio"

In [ ]:
from minio import Minio
client_minio = Minio(
    f"{minio_ip_address}:9000",
    access_key="root",
    secret_key="password",
    secure=False
)

# Création du bucket tp6
if client_minio.bucket_exists("tp6") == False:
    client_minio.make_bucket("tp6")



Remise à zéro : destruction des fichiers allData de Minio (si présents)

In [ ]:
# Clear directory
directory = "allData"
from minio.deleteobjects import DeleteObject
def deleteFolder1(bucketname, folderName):
    # Delete using "remove_objects"
    objects_to_delete = client_minio.list_objects(bucketname, prefix=folderName, recursive=True)
    objects_to_delete = [DeleteObject(x.object_name) for x in objects_to_delete]
    print(objects_to_delete)
    for del_err in client_minio.remove_objects(bucketname, objects_to_delete):
        print("Deletion Error: {}".format(del_err))
deleteFolder1("tp6", "allData")


Copie des fichiers allData dans [Minio](http://127.0.0.1:19001/tp7) (10 par seconde)

In [ ]:
import time, os
for f in os.listdir(directory):
    client_minio.fput_object("tp6", f"{directory}/{f}", f"{directory}/{f}")
    time.sleep(0.1)